## Imports

In [45]:
#%pip install python-Levenshtein
import pandas as pd
from bs4 import BeautifulSoup
import re
import Levenshtein
import json
import contractions
pd.set_option('display.max_colwidth', None)


In [46]:
with open('oyez.json') as f:
    data = json.load(f)


## Filter: Remove Data Irrelevant to Model Training

In [47]:
# Returns true if the given case's judgment has been decided, false otherwise
def is_decided(entry):
    for timeline in entry['timeline']:
        if timeline['event'] == "Decided":
            return True
    return False

# Returns true if the given case has the necessary columns for our analysis, false otherwise
# Currently only considers:
#    1- case's judgment has been decided
#    2- case has a non-empty facts field
#    3- case has a non-empty decisions field
#    4- case with the same first/second party
#    5- case has facts field with 4 or more words
#    6- case has a winning party
def is_entry_complete(entry):
    try:
        if not is_decided(entry):
            return False
        facts = entry['facts_of_the_case']
        if facts == None or len(str(facts)) == 0:
            return False
        # Ignore case with facts being Currently unknown, Not Available, Currently unavailable.
        if len(str(facts).split()) <= 3:
            return False
        #conclusion = entry['conclusion']
        #if conclusion == None or len(str(conclusion)) == 0:
        #    return False
        # Ignore case with facts being Currently unknown, Not Available, Currently unavailable.
        #if len(str(conclusion).split()) <= 3:
        #    return False
        if entry['decisions'] == None or len(entry['decisions']) == 0:
            return False
        if entry['first_party'] == entry['second_party'] and entry['first_party'] != None:
            return False
        for decision in entry['decisions']:
            winning_party = decision['winning_party']
            if winning_party == None or len(winning_party) <= 0 or str(winning_party).lower() in ['dismissal', 'n/a']:
                return False
        return True
    except:
        return False

# Returns a dict with only the necessary columns we need from a given case
# Currently the followings columns are considered:
#    1-  case ID (assigned by oyez.org)
#    2-  case name
#    3-  href URL to the oyez.org case
#    4-  name of the first party
#    5-  name of the second party
#    6-  name of the winning party
#    7-  facts of the case
#    8-  court's conclusion
def filter_entry(entry):
    row = {}
    row['ID'] = entry['ID']
    row['name'] = entry['name']
    row['first_party'] = entry['first_party']
    row['second_party'] = entry['second_party']
    row['winning_party'] = None
    for decision in entry['decisions']:
        winning_party = decision['winning_party']
        if winning_party != None and len(winning_party) > 0 and str(winning_party).lower() not in ['dismissal', 'n/a']:
            row['winning_party'] = winning_party
            break
    
    row['facts'] = entry['facts_of_the_case']
    row['question'] = entry['question']
    row['conclusion'] = entry['conclusion']
    return row


In [48]:
filtered_data = []
for entry in data:
    if is_entry_complete(entry):
        filtered_data.append(filter_entry(entry))
df = pd.DataFrame(filtered_data)
print(len(filtered_data))

2380


## Find winner_index

In [60]:
def is_abbreviation(abbreviation, full_string):
    cleaned_abbreviation = re.sub(r'\W+', '', abbreviation).upper()
    cleaned_full_string = re.sub(r'\W+', '', full_string).upper()
    it = iter(cleaned_full_string)
    return all(char in it for char in cleaned_abbreviation)

def closest_string(first_string, second_string, target_string):
    # Substring Check
    if target_string in first_string:
        return 0
    elif target_string in second_string:
        return 1
    #Abbreviation check
    if is_abbreviation(target_string, first_string):
        return 0
    elif is_abbreviation(target_string, second_string):
        return 1
    
    #Levenshtein fallback  
    distance_to_first = Levenshtein.distance(first_string, target_string)
    distance_to_second = Levenshtein.distance(second_string, target_string)

    if distance_to_first <= distance_to_second:
        return 0
    else:
        return 1

win_index = []
for index, row in df.iterrows():
    win_index.append(closest_string(row['first_party'], row['second_party'], row['winning_party']))
print(win_index)
df = df.assign(winner_index=win_index)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 

## Clean Data

In [61]:
def sanitize_review(text):
    # remove HTML tags
    text = BeautifulSoup(str(text), 'html.parser').get_text()   
    # remove URLS
    text = re.sub(r'http\S+', '', str(text))
    return text
def remove_non_ascii(text):
    return text.encode(encoding='utf-8', errors='ignore').decode()
def fix_contractions(text):
    return contractions.fix(text)

df['facts'] = df['facts'].apply(sanitize_review)
df['question'] = df['question'].apply(sanitize_review)
df['conclusion'] = df['conclusion'].apply(sanitize_review)

df['facts'] = df['facts'].str.replace('\n', ' ')
df['conclusion'] = df['conclusion'].str.replace('\n', ' ')
df['question'] = df['question'].str.replace('\n', ' ')

df['facts'] = df['facts'].apply(remove_non_ascii)
df['question'] = df['question'].apply(remove_non_ascii)
df['conclusion'] = df['conclusion'].apply(remove_non_ascii)

df['facts'] = df['facts'].apply(fix_contractions)
df['question'] = df['question'].apply(fix_contractions)
df['conclusion'] = df['conclusion'].apply(fix_contractions)


C:\Users\Nevin\AppData\Local\Temp\ipykernel_4508\4167049792.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(str(text), 'html.parser').get_text()


## Test DF

In [65]:
print(len(df))
df.sample(n=10)

2380


,ID,name,first_party,second_party,winning_party,facts,question,conclusion,winner_index
2229,63018,Nieves v. Bartlett,"Luis A. Nieves, et al.",Russell P. Bartlett,Nieves,"Russell Bartlett was arrested by Alaska state troopers Luis Nieves and Bryce Weight for disorderly conduct and harassment. Bartlett subsequently sued the officers for damages under 42 YOU.S.C. § 1983, making claims including false arrest and imprisonment, excessive force, malicious prosecution, and retaliatory arrest. The district court granted summary judgment to the officers on all claims. The YOU.S. Court of Appeals for the Ninth Circuit reversed the district court’s ruling on the retaliatory arrest claim, explaining that under its own precedent, a showing of probable because did not preclude a claim of retaliatory arrest. The appellate court noted that in 2012, the YOU.S. Supreme Court had clarified that its decision in Hartman v. Moore, 547 YOU.S. 250 (2006), which held that a plaintiff could not make a retaliatory prosecution claim if the charges were supported by probable because, did not necessarily extend to retaliatory arrests. And since that time, the Ninth Circuit had held that a plaintiff could make a retaliatory arrest claim even if the arresting officers had probable because.",Does probable because defeat a First Amendment retaliatory-arrest claim under 42 YOU.S.C. § 1983?,"The presence of probable because for an arrest defeats a First Amendment retaliatory arrest claim as a matter of law. Chief Justice John Roberts delivered the majority opinion. To prevail on a First Amendment retaliatory arrest claim, the plaintiff must show that the official acted with a retaliatory motive and that the motive was the “but-for” because of the plaintiff’s injury. The Court looked to analogous situations to determine how to identify whether improper motive caused the injury: the torts of false imprisonment and malicious prosecution. Analysis of motive of these torts supports the conclusion that the presence of probable because should defeat a retaliatory arrest claim, regardless of the subjective motive of the arresting officer. Thus, if the officer has probable because, then even the presence of a retaliatory motive motive is irrelevant unless the plaintiff presents “objective evidence that he was arrested when otherwise similarly situated individuals not engaged in the same sort of protected speech had not been” (an equal protection, rather than First Amendment, argument). Justice Clarence Thomas joined the majority opinion as to all but Part II-D (in which the Court described a narrow qualification for the situation in which officers have probable because for an arrest but exercise discretion not to do so). He wrote separately to concur in part and concur in the judgment. Justice Neil Gorsuch wrote an opinion concurring in part and dissenting in part. Justice Ruth Bader Ginsburg wrote an opinion concurring in the judgment in part and dissenting in part. Justice Sonia Sotomayor filed a dissenting opinion.",0
23,50816,Frontiero v. Richardson,Sharron Frontiero,"Elliot Richardson, Secretary of Defense",Frontiero,"Sharron Frontiero, a lieutenant in the United States Air Force, sought a dependent's allowance for her husband. Federal law provided that the wives of members of the military automatically became dependents; husbands of female members of the military, however, were not accepted as dependents unless they were dependent on their wives for over one-half of their support. Frontiero's request for dependent status for her husband was turned down.","Did a federal law, requiring different qualification criteria for male and female military spousal dependency, unconstitutionally discriminate against women thereby violating the Fifth Amendment's Due Process Clause?","Yes. The Court held that the statute in question clearly commanded ""dissimilar treatment for men and women who are similarly situated,"" violating the Due Process Clause and the equal protection req

## Prepare DF for Prediction Model

In [67]:
df.to_pickle('oyez_proccessed.pkl')